[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/neuralangelo-colab/blob/main/neuralangelo_colab.ipynb)

In [ ]:
%cd /content

!apt-get install \
    cmake \
    libgoogle-glog-dev \
    libgflags-dev \
    libatlas-base-dev \
    libeigen3-dev \
    libsuitesparse-dev \
    libboost-program-options-dev \
    libboost-filesystem-dev \
    libboost-graph-dev \
    libboost-system-dev \
    libboost-test-dev \
    libfreeimage-dev \
    libmetis-dev \
    libglew-dev \
    qtbase5-dev \
    libqt5opengl5-dev \
    libcgal-dev

!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/colmap.zip
!unzip /content/colmap.zip -d colmap

!cp -r /content/colmap/lib/. /usr/local/lib
!chmod 755 /content/colmap/bin/colmap
!cp -r /content/colmap/bin/. /usr/local/bin

!pip install -q commentjson addict pynvml wandb trimesh PyMCubes
!pip install -q https://huggingface.co/camenduru/CoDeF/resolve/main/tinycudann-1.7-cp310-cp310-linux_x86_64.whl

%cd /content
!git clone -b v1.0 --recursive https://github.com/camenduru/neuralangelo

In [ ]:
%cd /content/neuralangelo
!wget https://huggingface.co/camenduru/neuralangelo/resolve/main/lego.mp4 -O /content/lego.mp4
SEQUENCE="lego"
PATH_TO_VIDEO="/content/lego.mp4"
DOWNSAMPLE_RATE=2
SCENE_TYPE="object"
!bash /content/neuralangelo/projects/neuralangelo/scripts/preprocess.sh {SEQUENCE} {PATH_TO_VIDEO} {DOWNSAMPLE_RATE} {SCENE_TYPE}

In [ ]:
# from https://github.com/NVlabs/neuralangelo
# @title Visulize COLMAP (Optional)
!pip install -q k3d

colmap_path='/content/neuralangelo/datasets/lego_ds2'
%cd /content/neuralangelo
# Import Python libraries.
import numpy as np
import torch
import k3d
import json
import plotly.graph_objs as go
from collections import OrderedDict
# Import imaginaire modules.
from projects.nerf.utils import camera, visualize
from third_party.colmap.scripts.python.read_write_model import read_model
# Read the COLMAP data.
cameras, images, points_3D = read_model(path=f"{colmap_path}/sparse", ext=".bin")
# Convert camera poses.
images = OrderedDict(sorted(images.items()))
qvecs = torch.from_numpy(np.stack([image.qvec for image in images.values()]))
tvecs = torch.from_numpy(np.stack([image.tvec for image in images.values()]))
Rs = camera.quaternion.q_to_R(qvecs)
poses = torch.cat([Rs, tvecs[..., None]], dim=-1)  # [N,3,4]
print(f"# images: {len(poses)}")
# Get the sparse 3D points and the colors.
xyzs = torch.from_numpy(np.stack([point.xyz for point in points_3D.values()]))
rgbs = np.stack([point.rgb for point in points_3D.values()])
rgbs_int32 = (rgbs[:, 0] * 2**16 + rgbs[:, 1] * 2**8 + rgbs[:, 2]).astype(np.uint32)
print(f"# points: {len(xyzs)}")

# Visualize the bounding sphere.
json_fname = f"{colmap_path}/transforms.json"
with open(json_fname) as file:
    meta = json.load(file)
center = meta["sphere_center"]
radius = meta["sphere_radius"]
# ------------------------------------------------------------------------------------
# These variables can be adjusted to make the bounding sphere fit the region of interest.
# The adjusted values can then be set in the config as data.readjust.center and data.readjust.scale
readjust_x = 0.  # @param {type:"number"}
readjust_y = 0.  # @param {type:"number"}
readjust_z = 0.  # @param {type:"number"}
readjust_scale = 1.  # @param {type:"number"}
readjust_center = np.array([readjust_x, readjust_y, readjust_z])
# ------------------------------------------------------------------------------------
center += readjust_center
radius *= readjust_scale
# Make some points to hallucinate a bounding sphere.
sphere_points = np.random.randn(100000, 3)
sphere_points = sphere_points / np.linalg.norm(sphere_points, axis=-1, keepdims=True)
sphere_points = sphere_points * radius + center

vis_depth = 0.2
# Visualize with Plotly.
x, y, z = *xyzs.T,
colors = rgbs / 255.0
sphere_x, sphere_y, sphere_z = *sphere_points.T,
sphere_colors = ["#4488ff"] * len(sphere_points)
traces_poses = visualize.plotly_visualize_pose(poses, vis_depth=vis_depth, xyz_length=0.02, center_size=0.01, xyz_width=0.005, mesh_opacity=0.05)
trace_points = go.Scatter3d(x=x, y=y, z=z, mode="markers", marker=dict(size=1, color=colors, opacity=1), hoverinfo="skip")
trace_sphere = go.Scatter3d(x=sphere_x, y=sphere_y, z=sphere_z, mode="markers", marker=dict(size=0.5, color=sphere_colors, opacity=0.7), hoverinfo="skip")
traces_all = traces_poses + [trace_points, trace_sphere]
layout = go.Layout(scene=dict(xaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              yaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              zaxis=dict(showspikes=False, backgroundcolor="rgba(0,0,0,0)", gridcolor="rgba(0,0,0,0.1)"),
                              xaxis_title="X", yaxis_title="Y", zaxis_title="Z", dragmode="orbit",
                              aspectratio=dict(x=1, y=1, z=1), aspectmode="data"), height=800)
fig = go.Figure(data=traces_all, layout=layout)
fig.show()

In [ ]:
!wandb login

In [ ]:
%cd /content/neuralangelo
EXPERIMENT="lego"
GROUP="lego_group"
NAME="lego"
CONFIG=f"/content/neuralangelo/projects/neuralangelo/configs/custom/{EXPERIMENT}.yaml"
GPUS=1
CHECKPOINT_PATH="/content/checkpoint"
!torchrun --nproc_per_node={GPUS} train.py \
    --wandb \
    --wandb_name=neuralangelo \
    --logdir=logs/{GROUP}/{NAME} \
    --config={CONFIG} \
    --show_pbar \
    --data.readjust.scale=0.5 \
    --max_iter=25000 \
    --model.object.sdf.encoding.coarse2fine.step=200 \
    --model.object.sdf.encoding.hashgrid.dict_size=19 \
    --optim.sched.warm_up_end=200 \
    --optim.sched.two_steps=[12000,16000]

In [ ]:
%cd /content/neuralangelo
CONFIG=f"/content/neuralangelo/projects/neuralangelo/configs/custom/lego.yaml"
GPUS=1
CHECKPOINT="/content/neuralangelo/logs/lego_group/lego/epoch_00150_iteration_000015000_checkpoint.pt"
OUTPUT_MESH="/content/lego.ply"
RESOLUTION=2048
BLOCK_RES=128
!torchrun --nproc_per_node={GPUS} projects/neuralangelo/scripts/extract_mesh.py \
    --config={CONFIG} \
    --checkpoint={CHECKPOINT} \
    --output_file={OUTPUT_MESH} \
    --resolution={RESOLUTION} \
    --block_res={BLOCK_RES} \
    --textured